# Import Required Libraries

In [ ]:
import sys

# Data Manipulation libraries:
import numpy as np
import pandas as pd
from copy import deepcopy

# Plotting libraries
import mplsoccer
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

[mplsoccer docs](https://mplsoccer.readthedocs.io/en/latest/)

# Load the Data

In [ ]:
eventsDataLaLiga2019 = pd.read_csv("../data/matchwise_events_data_updated.csv",
                                   low_memory=False)

In [ ]:
seasonMetaDataLaLiga2019 = pd.read_csv("../data/season_matchwise_data.csv",
                                       low_memory=False)

In [ ]:
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 50)

In [ ]:
eventsDataLaLiga2019.columns

# Data Cleaning and Exploratory Data Analysis (EDA)

EDA, mainly, does three things:

1. It helps clean up a dataset.
1. It helps identify metrics that can be added by calculations.
2. It gives you a better understanding of the variables and the relationships between them.

## Data Prep - Separate out the pass columns

In [ ]:
eventPassdf = deepcopy(eventsDataLaLiga2019[eventsDataLaLiga2019["type.id"] == 30])

In [ ]:
eventPassdf.columns[eventPassdf.columns.str.startswith("pass")]

In [ ]:
eventPassdf["under_pressure"].unique()

In [ ]:
passAdditionalCols = ['match_id', 'id',
                      'timestamp', 'minute', 'second',
                      'type.id', 'type.name',
                      'possession_team.id', 'possession_team.name',
                      'play_pattern.id', 'play_pattern.name',
                      'team.id', 'team.name', 'location',
                      'player.id', 'player.name',
                      'position.id', 'position.name',
                      'under_pressure',
                      'started', 'minsPlayed']

In [ ]:
pass_cols = eventPassdf.columns[eventPassdf.columns.str.startswith("pass")].tolist()
pass_cols = passAdditionalCols + pass_cols
pass_cols

In [ ]:
eventPassdf = eventPassdf[pass_cols]

In [ ]:
eventPassdf.shape

In [ ]:
eventPassdf.nunique(axis=0)

## Separate Categorical and Numerical Columns

In [ ]:
eventPassdf.select_dtypes(include="category").columns

In [ ]:
eventPassdf.select_dtypes(include="object").columns

In [ ]:
catPassCols = eventPassdf.select_dtypes(include="object").columns
catPassCols

In [ ]:
unwantedCols = ["player.name", "pass.recipient.name",
                "pass.end_location", "location",
                "pass.assisted_shot_id", "id", "timestamp",
                "position.name", "possession_team.name",
                "team.name", "type.name"]
catPassCols.symmetric_difference(unwantedCols)

In [ ]:
catPassCols = catPassCols.symmetric_difference(unwantedCols)

In [ ]:
catPassCols.__len__()

In [ ]:
eventPassdf["pass.height.name"].value_counts()

In [ ]:
%timeit eventPassdf["pass.height.name"].value_counts()

In [ ]:
eventPassdf["pass.height.name"] = eventPassdf["pass.height.name"].astype("category")

In [ ]:
%timeit eventPassdf["pass.height.name"].value_counts()

In [ ]:
eventPassdf[catPassCols] = eventPassdf[catPassCols].astype("category")

In [ ]:
eventPassdf.select_dtypes(exclude=["object", "category"]).columns

In [ ]:
numPassCols = ['pass.length', 'pass.angle', 'minsPlayed']

## EDA on Pass Columns

### Numerical EDA

In [ ]:
eventPassdf.info()

In [ ]:
eventPassdf[numPassCols].describe().T.round(1)

In [ ]:
eventPassdf.isnull().sum()

In [ ]:
for col in eventPassdf[catPassCols]:
    print(col, "\n", eventPassdf[col].unique(), "\n")

In [ ]:
eventPassdf[["pass.outcome.id", "pass.outcome.name"]].drop_duplicates()

In [ ]:
eventPassdf["pass.outcome.id"] = np.where(eventPassdf["pass.outcome.id"].isnull(), 1,
                                          eventPassdf["pass.outcome.id"])
eventPassdf["pass.outcome.name"] = np.where(eventPassdf["pass.outcome.name"].isnull(), "Complete",
                                            eventPassdf["pass.outcome.name"])

In [ ]:
eventPassdf[["pass.outcome.id", "pass.outcome.name"]].drop_duplicates()

In [ ]:
booleanCols = ["under_pressure", "pass.shot_assist", "pass.aerial_won",
               "pass.deflected", "pass.switch", "pass.cross", "pass.through_ball",
               "pass.goal_assist", "pass.no_touch", "pass.outswinging",
               "pass.cut_back", "pass.inswinging", "pass.straight", "pass.miscommunication"]

In [ ]:
eventPassdf[booleanCols]

In [ ]:
eventPassdf[booleanCols].fillna(False)

In [ ]:
eventPassdf[booleanCols] = eventPassdf[booleanCols].fillna(False)

In [ ]:
eventPassdf["under_pressure"] = eventPassdf["under_pressure"].cat.add_categories(False)

In [ ]:
eventPassdf["under_pressure"].unique()

In [ ]:
eventPassdf["under_pressure"] = eventPassdf["under_pressure"].fillna(False)

In [ ]:
for col in booleanCols:
    try:
        eventPassdf[col] = eventPassdf[col].cat.add_categories(False)
    except ValueError:
        print(sys.exc_info())
    eventPassdf[col] = eventPassdf[col].fillna(False)    

In [ ]:
eventPassdf[booleanCols]

In [ ]:
for col in catPassCols:
    print(col, "\n", round(eventPassdf[col].value_counts(normalize=True)*100, 3), "\n")

### Visual EDA

In [ ]:
eventPassdf[numPassCols].hist(bins=10, figsize=(16, 8))

In [ ]:
from IPython.display import Image
Image("../pics/Pass Angle Explained.png")

In [ ]:
eventPassdf[catPassCols]

In [ ]:
eventPassdf["play_pattern.name"].value_counts().reset_index()

In [ ]:
catPassCols

In [ ]:
catPassCols.__len__()

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=5, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=catPassCols)
# Add bar plot for goals conceded in all subplots:
for col in catPassCols:
    plot_df = eventPassdf[col].value_counts().reset_index()
    fig.add_trace(go.Bar(x=plot_df["index"],
                         y=plot_df[col],
                         text=plot_df[col],
                         name=col),
                  row=i, col=j)
    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_traces(textposition='inside', textfont_size=10)
fig.update_layout(height=1600)

fig.show()

### Top Passing Teams

In [ ]:
teamWisedf = eventPassdf.groupby("team.id").agg({"team.name": "first", "match_id": "nunique", "type.id": "count"})

In [ ]:
teamWisedf.columns = ["teamName", "nMatches", "nPasses"]

In [ ]:
teamWisedf

In [ ]:
teamWisedf["passesPerMatch"] = teamWisedf["nPasses"].divide(teamWisedf["nMatches"])

In [ ]:
teamWisedf

In [ ]:
teamWisedf.style.highlight_max().set_precision(1)

In [ ]:
teamWisedf.style.bar().set_precision(1)

In [ ]:
teamWisedf.style.bar(subset=["passesPerMatch"]).set_precision(1)

In [ ]:
def color_avg_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val.dtype == "object":
        return [""]*len(val)
    valMean = val.mean()
    colors = ['color: green' if (v > valMean) else 'color: red' for v in val]
    return colors

In [ ]:
teamWisedf.style.apply(color_avg_red).set_precision(1)

### Top Passing Players

In [ ]:
playerWisedf = eventPassdf.groupby("player.id").agg({"player.name": "first", "team.name": "first",
                                                     "match_id": "nunique", "type.id": "count"})

In [ ]:
playerWisedf["passesPerMatch"] = playerWisedf["type.id"].divide(playerWisedf["match_id"])

In [ ]:
playerWisedf.style.apply(color_avg_red).set_precision(1)

In [ ]:
avgPassesPerMatch = playerWisedf["passesPerMatch"].mean()

In [ ]:
playerWisedf[playerWisedf["passesPerMatch"] > avgPassesPerMatch].style.apply(color_avg_red).set_precision(1)

In [ ]:
playerWisedf[playerWisedf["passesPerMatch"] > avgPassesPerMatch].style.highlight_max()

In [ ]:
playerWisedf[playerWisedf["passesPerMatch"] > avgPassesPerMatch].style.bar().set_precision(1)

### Top Passing Players - Per90

In [ ]:
eventPassdf.drop_duplicates(subset=["player.id", "match_id"]).groupby(["player.id"])\
.agg({"minsPlayed": "sum"})

In [ ]:
playerMatchMinsdf = eventPassdf.drop_duplicates(subset=["player.id", "match_id"]).groupby(["player.id"])\
.agg({"minsPlayed": "sum"})

In [ ]:
playerWisedf = eventPassdf.groupby("player.id").agg({"player.name": "first", "team.name": "first",
                                                     "match_id": "nunique", "type.id": "count"})

In [ ]:
playerWisedf.shape

In [ ]:
pd.concat([playerWisedf, playerMatchMinsdf], axis=1)

In [ ]:
playerWisedf = pd.concat([playerWisedf, playerMatchMinsdf], axis=1)

In [ ]:
playerWisedf["passesPerMatch"] = playerWisedf["type.id"].divide(playerWisedf["match_id"])

In [ ]:
playerWisedf["passesPer90"] = playerWisedf["type.id"].divide(playerWisedf["minsPlayed"])*90

In [ ]:
avgPassesPer90 = playerWisedf["passesPer90"].mean()

In [ ]:
playerWisedf[playerWisedf["passesPer90"] > avgPassesPer90].style.bar(subset=["passesPerMatch",
                                                                             "passesPer90"]).set_precision(1)

# Generating and Analysing Pass Maps

In [ ]:
eventPassdf.info()

In [ ]:
eventPassdf["location"].head()

In [ ]:
eventPassdf["pass.end_location"].head()

In [ ]:
eventPassdf["pass.end_location"].str.split(", ", expand=True)[1].str[:-1].apply(pd.to_numeric)

In [ ]:
eventPassdf["startX"] = eventPassdf["location"].str.split(", ", expand=True)[0].str[1:].apply(pd.to_numeric)
eventPassdf["startY"] = eventPassdf["location"].str.split(", ", expand=True)[1].str[:-1].apply(pd.to_numeric)

eventPassdf["endX"] = eventPassdf["pass.end_location"].str.split(", ", expand=True)[0].str[1:].apply(pd.to_numeric)
eventPassdf["endY"] = eventPassdf["pass.end_location"].str.split(", ", expand=True)[1].str[:-1].apply(pd.to_numeric)

In [ ]:
pitch = mplsoccer.Pitch(pitch_color='#000000', line_color='white')
fig, ax = pitch.draw(figsize=(14, 8))

In [ ]:
pitch = mplsoccer.Pitch(pitch_color='#101010', line_color='white')
fig, ax = pitch.draw(figsize=(14, 10))

arrows = pitch.arrows(eventPassdf["startX"], eventPassdf["startY"],
                      eventPassdf["endX"], eventPassdf["endY"],
                      ax=ax,
                      width=.1,
                      color="yellow")

In [ ]:
eventPassdf[["match_id", "team.name"]].drop_duplicates()

In [ ]:
matchEventdf = eventPassdf[eventPassdf["match_id"] == 303696]  # <-- Only addition to the code

pitch = mplsoccer.Pitch(pitch_color='#101010', line_color='white')
fig, ax = pitch.draw(figsize=(14, 10))

""" 
    | 
    v Only modification to the code
"""
arrows = pitch.arrows(matchEventdf["startX"], matchEventdf["startY"],
                      matchEventdf["endX"], matchEventdf["endY"],
                      ax=ax,
                      width=1,
                      color="green")

In [ ]:
team_ids = eventPassdf[eventPassdf["match_id"] == 303470]["possession_team.id"].unique()

In [ ]:
# DF for Team 1:
pass_map_df_team1 = eventPassdf[(eventPassdf["match_id"] == 303470)
                                & (eventPassdf["possession_team.id"] == team_ids[0])]
# DF for Team 2:
pass_map_df_team2 = eventPassdf[(eventPassdf["match_id"] == 303470)
                                & (eventPassdf["possession_team.id"] == team_ids[1])]

In [ ]:
pitch = mplsoccer.Pitch(pitch_color='#101010', line_color='white')
fig, ax = pitch.draw(figsize=(16, 8))

arrows = pitch.arrows(pass_map_df_team1["startX"], pass_map_df_team1["startY"],
                      pass_map_df_team1["endX"], pass_map_df_team1["endY"],
                      ax=ax,
                      width=1,
                      color="green")
ax.set_title(pass_map_df_team1["possession_team.name"].unique().item(), fontsize=30)

In [ ]:
pass_map_df_team2["team.name"].unique()

In [ ]:
pitch = mplsoccer.Pitch(pitch_color='#101010', line_color='white')
fig, ax = pitch.draw(figsize=(16, 8))

arrows = pitch.arrows(pass_map_df_team2["startX"], pass_map_df_team2["startY"],
                      pass_map_df_team2["endX"], pass_map_df_team2["endY"],
                      ax=ax,
                      width=1,
                      color="gold")
ax.set_title(pass_map_df_team2["possession_team.name"].unique().item(), fontsize=30)

## Side-by-Side Pass Maps

In [ ]:
""" Set Pitch Parameters """
pitch = mplsoccer.Pitch(pitch_color='#101010', line_color='white')

""" Create 1x2 grid of subplots for Analysing two teams """
fig, axs = pitch.draw(nrows=1, ncols=2, figsize=(16, 10))

""" Team 1 Pass Map """
# Draw arrows to create pass map
arrows = pitch.arrows(pass_map_df_team1["startX"],
                      pass_map_df_team1["startY"],
                      pass_map_df_team1["endX"],
                      pass_map_df_team1["endY"],
                      ax=axs[0],
                      width=1,
                      color="#ba4f45")
# Set title for subplot
axs[0].set_title(pass_map_df_team1["possession_team.name"].unique().item(),
                 fontsize=30)

""" Team 2 Pass Map """
# Draw arrows to create pass map
arrows = pitch.arrows(pass_map_df_team2["startX"],
                      pass_map_df_team2["startY"],
                      pass_map_df_team2["endX"],
                      pass_map_df_team2["endY"],
                      ax=axs[1],
                      width=1,
                      color="#ad993c")
# Set title for subplot
axs[1].set_title(pass_map_df_team2["possession_team.name"].unique().item(),
                 fontsize=30)

In [ ]:
"""
Create grouped data for every match
"""
groups = eventPassdf.groupby(["match_id"])

In [ ]:
for name, group in groups:
    print(name)
    print(group.head())

In [ ]:
"""
Go through each match and plot pass maps for each team
"""
for name, group in groups:
#     print(name)
    team_ids = group["possession_team.id"].unique()
    """ Bifurcate team-wise data """
#     print(group.head())
    pass_map_df_team1 = deepcopy(group[(group["possession_team.id"] == team_ids[0])])
    pass_map_df_team2 = deepcopy(group[(group["possession_team.id"] == team_ids[1])])

    """ Create 1x2 grid of subplots for Analysing two teams """
    fig, axs = pitch.draw(nrows=1, ncols=2, figsize=(16, 10))

    """ Team 1 Pass Map """
    # Draw arrows to create pass map
    arrows = pitch.arrows(pass_map_df_team1["startX"],
                          pass_map_df_team1["startY"],
                          pass_map_df_team1["endX"],
                          pass_map_df_team1["endY"],
                          ax=axs[0],
                          width=1,
                          color="#ba4f45")
    # Set title for subplot
    axs[0].set_title(pass_map_df_team1["possession_team.name"].unique().item(),
                     fontsize=30)

    """ Team 2 Pass Map """
    # Draw arrows to create pass map
    arrows = pitch.arrows(pass_map_df_team2["startX"],
                          pass_map_df_team2["startY"],
                          pass_map_df_team2["endX"],
                          pass_map_df_team2["endY"],
                          ax=axs[1],
                          width=1,
                          color="#ad993c")
    # Set title for subplot
    axs[1].set_title(pass_map_df_team2["possession_team.name"].unique().item(),
                     fontsize=30)
    
    fig.show()    

In [ ]:
"""
Save all the plots to a pdf
"""

my_pdf = PdfPages("../reports/passMapsLaLiga2019_20.pdf")

""" Go through each match and plot pass maps for each team """
for name, group in groups:
    print(name)
    team_ids = group["possession_team.id"].unique()
    """ Bifurcate team-wise data """
    print(group.head())
    pass_map_df_team1 = deepcopy(group[(group["possession_team.id"] == team_ids[0])])
    pass_map_df_team2 = group[(group["possession_team.id"] == team_ids[1])]

    """ Create 1x2 grid of subplots for Analysing two teams """
    fig, axs = pitch.draw(nrows=1, ncols=2, figsize=(16, 10))

    """ Team 1 Pass Map """
    # Draw arrows to create pass map
    arrows = pitch.arrows(pass_map_df_team1["startX"],
                          pass_map_df_team1["startY"],
                          pass_map_df_team1["endX"],
                          pass_map_df_team1["endY"],
                          ax=axs[0],
                          width=1,
                          color="#ba4f45")
    # Set title for subplot
    axs[0].set_title(pass_map_df_team1["possession_team.name"].unique().item(),
                     fontsize=30)

    """ Team 2 Pass Map """
    # Draw arrows to create pass map
    arrows = pitch.arrows(pass_map_df_team2["startX"],
                          pass_map_df_team2["startY"],
                          pass_map_df_team2["endX"],
                          pass_map_df_team2["endY"],
                          ax=axs[1],
                          width=1,
                          color="#ad993c")
    # Set title for subplot
    axs[1].set_title(pass_map_df_team2["possession_team.name"].unique().item(),
                     fontsize=30)
    
    fig.show()
    my_pdf.savefig()
    
my_pdf.close()

# Passing Matrix

### Barcelona

In [ ]:
eventPassdf[["team.id", "team.name"]].drop_duplicates()

In [ ]:
barcaData = eventPassdf[eventPassdf["team.id"] == 217]

In [ ]:
barcaData.pivot_table(values="type.id", index="player.id", columns="pass.recipient.id", aggfunc="count")

In [ ]:
barcaPassMatrix = barcaData.pivot_table(values="type.id", index="player.name",
                                        columns="pass.recipient.name", aggfunc="count")

In [ ]:
barcaPassMatrix.style.highlight_max(axis=0).set_precision(0)

In [ ]:
barcaPassMatrix.style\
    .background_gradient(cmap="Blues", axis=1)\
    .set_precision(0)

In [ ]:
barcaPassMatrix.style\
    .background_gradient(cmap="Blues")\
    .highlight_null('black').set_precision(0)

Go to [pandas styling page](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) and search for "Fun Stuff"

In [ ]:
barcaPassMatrix.style\
    .background_gradient(cmap="Blues", axis=1)\
    .highlight_null('black').set_precision(0)\
    .set_table_styles([dict(selector="th",
                            props=[("font-size", "4pt")]),
                       dict(selector="td",
                            props=[('padding', "0em 0em")]),
                       dict(selector="th:hover",
                            props=[("font-size", "12pt")]),
                       dict(selector="tr:hover td:hover",
                            props=[('max-width', '200px'),
                                   ('font-size', '15pt')])])

In [ ]:
plt.figure(figsize=(18, 12))
sns.heatmap(data=barcaPassMatrix, annot=True, linewidths=1, fmt=".0f", cmap="viridis")
plt.xlabel("")
plt.ylabel("")

NOTE: You can find more colormaps (for argument `cmap`) at the [seaborn colormap gallery](https://seaborn.pydata.org/tutorial/color_palettes.html)

### El-Classico

In [ ]:
eventPassdf.loc[eventPassdf["team.name"].str.contains("Madrid"), ["team.name", "team.id"]].drop_duplicates()

In [ ]:
seasonMetaDataLaLiga2019[["season.season_id", "season.season_name"]].drop_duplicates()

In [ ]:
seasonMetaDataLaLiga2019[(seasonMetaDataLaLiga2019["season.season_id"] == 42) &
                         (seasonMetaDataLaLiga2019["home_team.home_team_id"].isin([217, 220])) &
                         (seasonMetaDataLaLiga2019["away_team.away_team_id"].isin([217, 220]))]

In [ ]:
eventPassdf[eventPassdf["match_id"].isin([303596, 303470])]["match_id"].drop_duplicates()

In [ ]:
eventPassdf[eventPassdf["match_id"].isin([303596, 303470])]["match_id"].drop_duplicates()

In [ ]:
elClassicoData = eventPassdf[eventPassdf["match_id"].isin([303596, 303470])]

In [ ]:
elClassicoData.pivot_table(values="type.id", index="player.name", columns="pass.recipient.name", aggfunc="count")

In [ ]:
elClassicoPassMatrix = elClassicoData.pivot_table(values="type.id", index="player.name",
                                                  columns="pass.recipient.name", aggfunc="count")

In [ ]:
elClassicoPassMatrix.style\
    .background_gradient(cmap="Blues", axis=1)\
    .highlight_null('black').set_precision(0)\
    .set_table_styles([dict(selector="th",
                            props=[("font-size", "4pt")]),
                       dict(selector="td",
                            props=[('padding', "0em 0em")]),
                       dict(selector="th:hover",
                            props=[("font-size", "12pt")]),
                       dict(selector="tr:hover td:hover",
                            props=[('max-width', '200px'),
                                   ('font-size', '15pt')])])

In [ ]:
plt.figure(figsize=(18, 12))
sns.heatmap(data=elClassicoPassMatrix, annot=True, linewidths=1, fmt=".0f", cmap="YlGnBu")
plt.xlabel("")
plt.ylabel("")

In [ ]:
plt.figure(figsize=(18, 12))
sns.heatmap(data=elClassicoPassMatrix[elClassicoPassMatrix.apply(lambda x: x > 10)],
            annot=True, linewidths=1, fmt=".0f", cmap="YlGnBu")
plt.xlabel("")
plt.ylabel("")